In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #Data Visualization 
import seaborn as sns  #Python library for Vidualization
from sklearn.cluster import KMeans

max = 9223372036854775807

In [2]:
# defining calculation functions

def calcBA(pruns, pinnings, pnotout):
    ba = pruns / (pinnings - pnotout)
    return ba


def calcBS(pruns, pbfaced):
    bs = pruns / pbfaced
    return bs


def calcMRA(pinnings, pnotout, pbfaced):
    mra = (pinnings - pnotout) / pbfaced
    return mra


def calcBRPI(pfours, psixes, pinnings):
    brpi = (4*pfours + 6*psixes) / pinnings
    return brpi


def calcBatOutRate(pinnings, pnotout, pbfaced):
    outrate = pd.Series(np.array([max])) if float(pbfaced) == 0 else (pinnings - pnotout) / pbfaced
    return outrate

#         BA = calcBA(p['runs'], p['innings'], p['not_out'])
#         BS = calcBS(p['runs'], p['ball_faced'])
#         MRA = calcMRA(p['innings'], p['not_out'], p['ball_faced'])
#         BRPI = calcBRPI(p['fours'], p['sixes'], p['innings'])


def calcGenBatAvg(truns, twickets):
    return truns / twickets


def calcGenBatOutRate(twickets, tbfaced):
    return twickets / tbfaced


def calcGenStrikeRate(truns, tbfaced):
    return truns / tbfaced


def calcBatAGR(tpRuns, tpBallfaced, tpOutRate, gBatAvg, gStrikeRate, gOutRate):
    agr = ((tpRuns - gStrikeRate * tpBallfaced) + gBatAvg * tpBallfaced * (gOutRate - tpOutRate))
#     d = np.array([132])
#     tm = pd.Series(np.array([0]))
    return agr


def calcBatRankingIndex(agr, gBatAvg):
    return agr / (10 * gBatAvg)


In [3]:
# defining bpwlling related calculation functions

# for each player

def calcBowlAvg(runsGiven, wicketsTaken):
    return runsGiven / wicketsTaken


def calcBowlSR(ballsBowled, wicketsTaken):
    return ballsBowled / wicketsTaken


def calcBowlER(runsGiven, oversBowled):
    return runsGiven / oversBowled


def calcBowlOutRate(wicketsTaken, ballsBowled):
    outr = pd.Series(np.array([0])) if float(ballsBowled) == 0 else wicketsTaken / ballsBowled
    return outr

# generic tournament parameters

def calcGenBowlAvg(totalRunsGiven, totalWicketsTaken):
    return totalRunsGiven / totalWicketsTaken


def calcGenBowlOutRate(totalWicketsTaken, totalOvers):
    balls = int(totalOvers) * 6 + 10 * (totalOvers - int(totalOvers))
    return totalWicketsTaken / balls


def calcGenBowlSR(totalRunsGiven, totalOvers):
    balls = int(totalOvers) * 6 + 10 * (totalOvers - int(totalOvers))
    return  totalRunsGiven / balls


def calcBowlAGR(tpRunsGiven, tpOvers, tpOutrate, gBowlAvg, gBowlSR, gBowlOutrate ):
    balls = int(tpOvers) * 6 + 10 * (tpOvers - int(tpOvers))
    bowlagr = (gBowlSR * balls - tpRunsGiven) + gBowlAvg * balls * (tpOutrate - gBowlOutrate)
    
#     print(bowlagr[0])
    if tpOvers[0] == 0.1:
        print('-------------------------------')
        print('tpRunsGiven  :',tpRunsGiven[0]) 
        print('tpballs      :', balls[0])
        print('tpOutrate    :', tpOutrate[0])
        print('gBowlAvg     :', gBowlAvg[0]) 
        print('gBowlSR      :', gBowlSR[0])
        print('gBowlOutrate :',  gBowlOutrate[0])
        print('-------------------------------')
        print('bowlAGR      : ', bowlagr)
        
    return bowlagr


def calcBowlRI(bowlAGR, gBowlAvg):
    bri = bowlAGR / (10 * gBowlAvg)
    
    if bri[0] == 0:
        print('-------------------------------')
        print('bowlagr  : ', bowlAGR[0] )
        print('gbowlAvg : ', gBowlAvg[0])
        print('-------------------------------')
    
    return bowlAGR / (10 * gBowlAvg)

In [4]:
df = pd.read_csv("serieswiseMatchRecord.csv")
print(df.columns)

Index(['series_code', 'match_code', 'pcid', 'pcountry', 'pid', 'name',
       'notout', 'runs', 'ballfaced', 'fours', 'sixes', 'overs',
       'maiden_overs', 'runs_given', 'wickets_taken'],
      dtype='object')


In [5]:
df['ballfaced'].replace({0 : 1}, inplace=True)
df['overs'].replace({0.0 : 0.1}, inplace=True)
# df[df['ballfaced'] == 1]

In [6]:
series = list(dict.fromkeys(df["series_code"]).keys())
# series.sort(reverse=True)

# trecord : dataframe for tournament overall performance
# precord : dataframe for player specific performance in tournament
# tplayer: dataframe for recording total runs, ballfaced etc for player

# seriespdata = pd.DataFrame()
trecord = pd.DataFrame()
precord = pd.DataFrame()
tplayer = pd.DataFrame()
pindexes = pd.DataFrame()

tournamentPerformance = dict()

for sr in series[:]:
    tdf = df[df["series_code"] == sr]
 
    ttotal = tdf.drop(['series_code', 'match_code', 'pid', 'name', 'pcid',
                                               'pcountry'], axis = 1)
    
    ttotal = pd.DataFrame(ttotal.sum(axis = 0))
    ttotal = ttotal.swapaxes('index', 'columns')
    ttotal.insert(0, 'series_code', value = sr)
    
    gBatAvg = calcGenBatAvg(ttotal['runs'], ttotal['wickets_taken'])
    gBatOutRate = calcGenBatOutRate(ttotal['wickets_taken'], ttotal['ballfaced'])
    gStrikeRate = calcGenStrikeRate(ttotal['runs'], ttotal['ballfaced'])
    
    gBowlAvg = calcBowlAvg(ttotal['runs_given'], ttotal['wickets_taken'])
    gBowlOutrate = calcGenBowlOutRate(ttotal['wickets_taken'], ttotal['overs'])
    gBowlSR = calcGenBowlSR(ttotal['runs_given'], ttotal['overs'])
    
    tp = {
        'series_code' : sr, 
        'gBatAvg'     : gBatAvg[0], 
        'gBatOutRate' : gBatOutRate[0], 
        'gStrikeRate' : gStrikeRate[0],
        'gBowlAvg'    : gBowlAvg[0], 
        'gBowlOutrate' : gBowlOutrate[0], 
        'gBowlSR'     : gBowlSR[0]
    }
    
    trecord = trecord.append(tp, ignore_index=True)
    
    
#     print((tp))
# print(len(series))

    players = list()
    for pl in set(tdf["pid"]):
        if pl > 0:
            players.append(pl)
        
#     print('current ',players)
    
    for player in players[:]:
        
        
        ptdf = tdf[tdf['pid'] == player].drop(['series_code', 'match_code', 'pid', 'name', 'pcid',
                                               'pcountry'], axis = 1)
        
        pinnings = len(ptdf)
        p = pd.DataFrame(ptdf.sum(axis = 0))
        p = p.swapaxes('index', 'columns')
        
#         print(p)
        tplayer = tplayer.append(p)
    
        batOutRate = calcBatOutRate(pinnings, p['notout'], p['ballfaced'])
        bowlOutRate = calcBowlOutRate(p['wickets_taken'], p['overs'])
        
#         p.insert(0, 'innings' ,value = pinnings)
#         p.insert(0, 'pid' ,value = player)
#         p.insert(0, 'series_code' ,value = sr)
#         p.insert(4, 'outrate' ,value = outRate)

        batAGR = calcBatAGR(p['runs'], p['ballfaced'], batOutRate, gBatAvg, gStrikeRate, gBatOutRate)
        batRI = calcBatRankingIndex(batAGR, gBatAvg)
        
        bowlAGR = calcBowlAGR(p['runs_given'], p['overs'], bowlOutRate, gBowlAvg, gBowlSR, gBowlOutrate)
        bowlRI = calcBowlRI(bowlAGR, gBowlAvg)
        
        pRI = {
            'series_code' : sr,
            'pid'    : player,
            'batAGR' : batAGR[0],
            'batRI'  : batRI[0],
            'bowlAGR': bowlAGR[0],
            'bowlRI' : bowlRI[0]
        }
        
        pindexes = pindexes.append(pRI, ignore_index = True)
#         print(type(batAGR))
#         print(batOutRate[0], bowlOutRate[0])
#         print('batAGR ', batAGR[0], '                        batRI ', batRI[0])
#         print('bowlAGR ', bowlAGR[0], '                       bowlRI ', bowlRI[0])
#         print()
        

-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  26.177777777777777
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  32.1025641025641
-------------------------------
---

-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 26.84
gBowlSR      : 0.8314745972738542
gBowlOutrate : 0.03097893432465925
-------------------------------
bowlAGR      :  0   -1.110223e-16
dtype: float64
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 27.0
gBowlSR      : 0.818181818181818
gBowlOutrate : 0.030303030303030293
-------------------------------
bowlAGR      :  0    1.110223e-16
dtype: float64
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 27.0
gBowlSR      : 0.818181818181818
gBowlOutrate : 0.030303030303030293
-------------------------------
bowlAGR      :  0    1.110223e-16
dtype: float64
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 27.0
gBowlSR      : 0.818181818181818
gBowlOutrate : 0.030303030303030293
-------------------------------
bowlAGR     

-------------------------------
bowlagr  :  0.0
gbowlAvg :  20.870967741935484
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  20.870967741935484
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  20.870967741935484
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 20.870967741935484
gBowlSR      : 0.5023291925465843
gBowlOutrate : 0.024068322981366477
-------------------------------
bowlAGR      :  0    0.0
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  20.870967741935484
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  20.870967741935484
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  20.870967741935484
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0

dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  32.935483870967744
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  32.935483870967744
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 32.935483870967744
gBowlSR      : 0.6708278580814726
gBowlOutrate : 0.02036793692509858
-------------------------------
bowlAGR      :  0    0.0
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  32.935483870967744
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 32.935483870967744
gBowlSR      : 0.6708278580814726
gBowlOutrate : 0.02036793692509858
-------------------------------
bowlAGR      :  0    0.0
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  32.935483870967744
-------------------------------
-------------

-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.96774193548387
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.96774193548387
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.96774193548387
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.96774193548387
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.96774193548387
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  33.371428571428574
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  33.371428571428574
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  33.371428571428574
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  33.371428571428574
------

...

-------------------------------
bowlagr  :  0.0
gbowlAvg :  44.45762711864407
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  44.45762711864407
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  31.41025641025641
-------------------------------
----------

dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  27.7
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 27.7
gBowlSR      : 0.628593040847202
gBowlOutrate : 0.022692889561270833
-------------------------------
bowlAGR      :  0    0.0
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  27.7
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  27.7
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  27.7
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 43.95161290322581
gBowlSR      : 0.9537976898844928
gBowlOutrate : 0.021701085054252677
-------------------------------
bowlAGR      :  0    1.110223e-16
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  43.95161290322

-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  28.84375
----------------------

-------------------------------
bowlagr  :  0.0
gbowlAvg :  39.328125
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  39.328125
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 30.125
gBowlSR      : 0.8092679650772338
gBowlOutrate : 0.02686366689053058
-------------------------------
bowlAGR      :  0    0.0
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.125
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 30.125
gBowlSR      : 0.8092679650772338
gBowlOutrate : 0.02686366689053058
-------------------------------
bowlAGR      :  0    0.0
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.125
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  30.125
---------------------------

-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  54.38235294117647
-------------------------------
-------------------------------
tpRunsGiven  : 0.0
tpballs      : 1.0
tpOutrate    : 0.0
gBowlAvg     : 48.08571428571429
gBowlSR      : 0.9762180974477961
gBowlOutrate : 0.0203016241299304
-------------------------------
bowlAGR      :  0   -1.110223e-16
dtype: float64
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.588235294117645
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.588235294117645
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.588235294117645
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.588235294117645
-------------------------------
-------------------------------
bowlagr  :  0.0
gbowlAvg :  52.588235294117645
-------------------------------
-------------------------------


In [7]:
pindexes.head()

,batAGR,batRI,bowlAGR,bowlRI,pid,series_code
0,-55.4,-0.206716,0.000000,0.000000,1794.0,191.0
1,-47.6,-0.177612,47.066667,0.179796,2180.0,191.0
2,32.4,0.120896,707.333333,2.702037,2191.0,191.0
3,110.4,0.411940,515.266667,1.968336,791.0,191.0
4,13.0,0.048507,114.851406,0.438736,29.0,191.0


In [8]:
pindexes.to_csv('playerIR.csv', index = False)
